In [1]:
import lightkurve as lk
import pandas as pd
import phasefolder
import os

%load_ext autoreload
%autoreload 2

In [13]:
def csv_to_lc(file):
    lcdata = pd.read_csv(file)
    lc = lk.LightCurve(time=lcdata.time, 
                       flux=lcdata.flux,
                       flux_err=lcdata.flux_err,
                       quality=lcdata.quality)
    return lc

In [15]:
lcc = [csv_to_lc("lightcurves/"+file) for file in os.listdir("lightcurves")]

In [18]:
from ipywidgets import interactive, widgets
import matplotlib.pyplot as plt
def f(a, b, c):
    lc = lcc[a]
    # best_period = fold(lc)
    folded_lc = lc.fold(b, normalize_phase=False, epoch_time=c)

    fig, ax = plt.subplots(2, figsize=(10,4), tight_layout=True)
    lc.scatter(ax=ax[0])
    folded_lc.scatter(ax=ax[1], label=f"Period: {b:.3f} d");
    plt.show()
    return fig

def update_best_guess(*args):
    lc = lcc[a.value]
    best_period = phasefolder.find_period(lc)
    b.value = best_period

a = widgets.BoundedIntText(
    value=0,
    min=0,
    max=len(lcc),
    step=1,
    description='Index:',
    disabled=False
)
b = widgets.BoundedFloatText(
    value=5,
    min=0.1,
    max=15,
    step=0.005,
    description='Period:',
    disabled=False
)
c = widgets.BoundedFloatText(
    value=min(lcc[a.value].time.value),
    min=min(lcc[a.value].time.value),
    max=max(lcc[a.value].time.value),
    step=1,
    description='Epoch Time:',
    disabled=False
)
d = widgets.Dropdown(
    options=['0.5', '2', '3'],
    value='0.5',
    description='Factor:',
    disabled=False,
)
button = widgets.Button(
    description='Save',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save',
)
e = widgets.Button(
    description=str(float(d.value)*b.value),
    disabled=False,
    button_style='',
    tooltip="factor*period"
)

def on_button_clicked(button):
    label = lcc[a.value].label
    fig = f(a.value, b.value, c.value)
    fig.savefig(f"{label}.png")
    folded_lc = lc.fold(b.value, normalize_phase=False)
    folded_lc.to_csv(f"{label}.csv", overwrite=True);

def on_period_change(change):
    new_period = change.new
    e.description = str(new_period*float(d.value))
    
def on_factor_change(change):
    new_factor = change.new
    e.description = str(b.value*float(new_factor))

def factor_button(button):
    b.value = float(e.description)
    
a.observe(update_best_guess, 'value')
b.observe(on_period_change, 'value')
d.observe(on_factor_change, 'value')
update_best_guess()
d.value = '2'
button.on_click(on_button_clicked)
e.on_click(factor_button)
out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(a, b, d, e, c, out, button)

BoundedIntText(value=0, description='Index:', max=10)

BoundedFloatText(value=2.187303177812704, description='Period:', max=15.0, min=0.1, step=0.005)

Dropdown(description='Factor:', index=1, options=('0.5', '2', '3'), value='2')

Button(description='4.374606355625408', style=ButtonStyle(), tooltip='factor*period')

BoundedFloatText(value=1325.328073571648, description='Epoch Time:', max=1353.1615065093147, min=1325.32807357…

Output()

Button(description='Save', style=ButtonStyle(), tooltip='Save')